In [81]:
import cvxpy as cp
import numpy as np
from itertools import product

In [82]:
shipping_cost = np.array([[1,3,2],[3,2,2]]).reshape(2,3)
given_demand = np.array([[100,150,120],[120,180,150],[150,200,180]]).reshape(3,3)
selling_price = np.array([[16,16]]).reshape(2,1)
produce = np.array([100,200]).reshape(2,1)
production_cost = np.array([11,10]).reshape(2,1)
produce_limit = np.array([300,300]).reshape(2,1)
senario = 27
senario_prob = (1/senario)*(np.ones((senario,1)))

In [87]:
senario = np.array([[100,120,150],
       [150,180,200],[120,150,180]])
senario_comb = []
index = np.arange(0,3)
index_permute = product(index,repeat=3)
for i in index_permute:
    temp_array =[]
    for j in range(0,3):
        temp_array.append(float(senario[j,:][i[j]]))
    senario_comb.append(temp_array)
all_demand = np.array(senario_comb).reshape(27,3)

In [88]:
all_demand

array([[100., 150., 120.],
       [100., 150., 150.],
       [100., 150., 180.],
       [100., 180., 120.],
       [100., 180., 150.],
       [100., 180., 180.],
       [100., 200., 120.],
       [100., 200., 150.],
       [100., 200., 180.],
       [120., 150., 120.],
       [120., 150., 150.],
       [120., 150., 180.],
       [120., 180., 120.],
       [120., 180., 150.],
       [120., 180., 180.],
       [120., 200., 120.],
       [120., 200., 150.],
       [120., 200., 180.],
       [150., 150., 120.],
       [150., 150., 150.],
       [150., 150., 180.],
       [150., 180., 120.],
       [150., 180., 150.],
       [150., 180., 180.],
       [150., 200., 120.],
       [150., 200., 150.],
       [150., 200., 180.]])

In [84]:
def solve_second_stage_problem(produce,demand,shipping_cost):

    sell = cp.Variable(shape=(2,3))
    salvage = cp.Variable(shape=(2,1))
    
    objective = cp.Minimize(-16*cp.sum(sell,keepdims=True) + cp.trace((((shipping_cost@(sell.T))))))
    constraints = [cp.sum(sell,axis=1,keepdims=True) +salvage <= produce,cp.sum(sell,axis=0,keepdims=True) <= demand.reshape(1,-1), sell>= np.zeros_like(sell),salvage>= np.zeros_like(salvage)]

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return problem

In [85]:
def solve_master_problem(produce,produce_limit,g_ks,alpha_ks): 

    production_cost = np.array([11,10]).reshape(2,1)
    produce  = cp.Variable(shape=(2,1),name="produce")
    v = cp.Variable(shape=(1,1),name="value")
   
    constraints = []
    for i in range(0,len(g_ks)):
        constraints.append(np.array(g_ks[i]).T@produce+np.array(alpha_ks[i])<=v)

    constraints.extend([produce>=0,v>=-100000,produce<=produce_limit])

    objective = cp.Minimize(production_cost.T@produce + v)

    problem = cp.Problem(objective, constraints)
    problem.solve()

    return problem

In [86]:
g_ks = []
alpha_ks = []

objctive_values  = [np.nan]
epsilon = 10**(-4)
iter = 0

while True:

    # Solve Second stage problem for each demand and store its duals and objective values
    duals = []
    objs= []
    for demand in all_demand:

        second_stage_sol = solve_second_stage_problem(produce,demand,shipping_cost)

        temp_dual = second_stage_sol.constraints[0].dual_value   # Take the duals of 1st contraint
        temp_obj = second_stage_sol.value                        # Take the objective value of second stage problem

        # Store duals and objective values for each senario
        duals.append(temp_dual) 
        objs.append(temp_obj)

    # Reshaping the values 
    duals = np.array(duals).reshape(-1,2)
    objs = np.array(objs).reshape(-1,1)

    g_ks_temp = (-senario_prob.T@duals).T
    alpha_ks_temp = senario_prob.T@objs - g_ks_temp.T@produce

    g_ks.append(g_ks_temp)
    alpha_ks.append(alpha_ks_temp)

    first_stage_sol = solve_master_problem(produce,produce_limit,g_ks,alpha_ks)
    obj_value = first_stage_sol.value
    new_produce = first_stage_sol.var_dict["produce"].value
    new_limit = first_stage_sol.var_dict["value"].value

    if np.abs(obj_value - objctive_values[-1])<= epsilon:
        print("Terminating condition satisfied !")
        break
    else:
        pass

    objctive_values.append(obj_value)
    produce,limit = new_produce,new_limit # swap the values
    iter = iter+1
    
    print(f"\n----------Iteration no.  {iter}--------------------")
    print(f"\nproduction is {produce}")
    print(f"\nobjctive value is {first_stage_sol.value}\n")



----------Iteration no.  1--------------------

production is [[300.00000004]
 [299.99999992]]

objctive value is -2351.895943550797


----------Iteration no.  2--------------------

production is [[149.00805118]
 [299.9999992 ]]

objctive value is -1784.2447749212179


----------Iteration no.  3--------------------

production is [[112.70470909]
 [300.00000003]]

objctive value is -1647.7631011106232


----------Iteration no.  4--------------------

production is [[125.23365091]
 [299.99999495]]

objctive value is -1618.5289057328064


----------Iteration no.  5--------------------

production is [[119.99999869]
 [299.99999991]]

objctive value is -1609.9999959895367

Terminating condition satisfied !
